## Aim: combine my work with solution

There are some parts of the Benson Solutions I like, and some parts where I like my approach better. I'm going to attempt to blend the two here

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


import sys
sys.path.append('./code')
import get_mta_data as gmd
import process_mta_data as pmd

# import datetime
from datetime import datetime as dt

## import data

In [2]:
mta = gmd.load_local_data([2019], [4])

## Tidy up data

The functions that are becoming standard for us.

In [3]:
mta = pmd.clean_col_names(mta)

In [4]:
mta = pmd.add_datetime(mta)

In [5]:
mta = pmd.convert_date_to_datetime(mta)



For convenience, add a 'turnstile_id' column

In [6]:
mta['turnstile_id'] = mta.CA + mta.UNIT + mta.SCP

Find and remove duplicate values

In [7]:
mta.DESC.value_counts()

REGULAR       747615
RECOVR AUD      3621
Name: DESC, dtype: int64

There are 3621 rows with 'RECOVER AUD'. In Lara's presentation, these were shown to be be duplicates in at least some cases. We will remove them all.

In [8]:
mta = mta[mta.DESC != 'RECOVR AUD']
mta.DESC.value_counts()

REGULAR    747615
Name: DESC, dtype: int64

This confirms that the 'RECOVER AUD' entries have been removed

Here's what the data looks like now:

In [9]:
mta.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime,turnstile_id
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,2019-03-30 00:00:00,A002R05102-00-00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,2019-03-30 04:00:00,A002R05102-00-00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,2019-03-30 08:00:00,A002R05102-00-00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,2019-03-30 12:00:00,A002R05102-00-00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,2019-03-30 16:00:00,A002R05102-00-00


Let's check datetime for value counts

In [10]:
mta.datetime.dt.time.value_counts()

16:00:00    63450
04:00:00    63418
00:00:00    63394
20:00:00    63370
08:00:00    63331
            ...  
16:52:32        1
14:45:15        1
20:09:35        1
09:23:58        1
06:58:50        1
Name: datetime, Length: 30247, dtype: int64

This shows that there are some entries at odd times. There appear to be more at standardized times.  
So, let's isolate those at 00:00:00 time

In [11]:
mask = mta['datetime'].dt.time == dt(2016, 1, 1, 0, 0, 0).time() # date is arbitrary, important part is time
mta = mta[mask]
mta.datetime.dt.time.value_counts()

00:00:00    63394
Name: datetime, dtype: int64

This confirms that all entries are now at midnight

Now, check that "CA", "UNIT", "SCP", "STATION", "datetime" is unique


In [12]:
mta_groups_1 = (mta
 .groupby(["CA", "UNIT", "SCP", "STATION", "datetime"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False))
mta_groups_1.ENTRIES

0        1
42252    1
42254    1
42255    1
42256    1
        ..
21136    1
21137    1
21138    1
21139    1
63393    1
Name: ENTRIES, Length: 63394, dtype: int64

Okay, that shows that the rows are all unique in terms of "CA", "UNIT", "SCP", "STATION", and "datetime".

Let's check that turnstile_id groupby gives the same result

In [13]:
mta_groups_2 = (mta
    .groupby(['turnstile_id', 'datetime'])
    ['ENTRIES'].count()
    .reset_index()
    .sort_values("ENTRIES", ascending=False))
mta_groups_2.ENTRIES

0        1
42252    1
42254    1
42255    1
42256    1
        ..
21136    1
21137    1
21138    1
21139    1
63393    1
Name: ENTRIES, Length: 63394, dtype: int64

In [14]:
assert (mta_groups_2.ENTRIES == mta_groups_1.ENTRIES).all()

The assertion passed, confirming that those two are the same, so I can use them equivalently

In [15]:
turnstiles_daily = (mta.groupby(["CA", "UNIT", "SCP", "STATION", "DATE"])
                    ['ENTRIES', 'datetime', 'turnstile_id'].first()).reset_index()

In [16]:
turnstiles_daily

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,turnstile_id
0,A002,R051,02-00-00,59 ST,2019-03-30,6999064,2019-03-30,A002R05102-00-00
1,A002,R051,02-00-00,59 ST,2019-03-31,6999957,2019-03-31,A002R05102-00-00
2,A002,R051,02-00-00,59 ST,2019-04-01,7000528,2019-04-01,A002R05102-00-00
3,A002,R051,02-00-00,59 ST,2019-04-02,7002087,2019-04-02,A002R05102-00-00
4,A002,R051,02-00-00,59 ST,2019-04-03,7003680,2019-04-03,A002R05102-00-00
...,...,...,...,...,...,...,...,...
63389,S101A,R070,01-05-01,ST. GEORGE,2019-04-15,625,2019-04-15,S101AR07001-05-01
63390,S101A,R070,01-05-01,ST. GEORGE,2019-04-16,626,2019-04-16,S101AR07001-05-01
63391,S101A,R070,01-05-01,ST. GEORGE,2019-04-17,626,2019-04-17,S101AR07001-05-01
63392,S101A,R070,01-05-01,ST. GEORGE,2019-04-18,626,2019-04-18,S101AR07001-05-01


Add columns for previous date and previous entries

In [17]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES", 'datetime']] = (turnstiles_daily
                                                       .groupby(["CA", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES", 'datetime']
                                                       .transform(lambda grp: grp.shift(1)))


And add a today - prev column

In [18]:
turnstiles_daily['day_minus_prev_entries'] = (turnstiles_daily.ENTRIES - turnstiles_daily.PREV_ENTRIES)

In [19]:
turnstiles_daily.head()

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,turnstile_id,PREV_DATE,PREV_ENTRIES,day_minus_prev_entries
0,A002,R051,02-00-00,59 ST,2019-03-30,6999064,NaT,A002R05102-00-00,NaT,NaN,NaN
1,A002,R051,02-00-00,59 ST,2019-03-31,6999957,2019-03-30,A002R05102-00-00,2019-03-30,6999064.0,893.0
2,A002,R051,02-00-00,59 ST,2019-04-01,7000528,2019-03-31,A002R05102-00-00,2019-03-31,6999957.0,571.0
3,A002,R051,02-00-00,59 ST,2019-04-02,7002087,2019-04-01,A002R05102-00-00,2019-04-01,7000528.0,1559.0
4,A002,R051,02-00-00,59 ST,2019-04-03,7003680,2019-04-02,A002R05102-00-00,2019-04-02,7002087.0,1593.0


In [20]:
turnstiles_daily.tail()

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,turnstile_id,PREV_DATE,PREV_ENTRIES,day_minus_prev_entries
63389,S101A,R070,01-05-01,ST. GEORGE,2019-04-15,625,2019-04-14,S101AR07001-05-01,2019-04-14,625.0,0.0
63390,S101A,R070,01-05-01,ST. GEORGE,2019-04-16,626,2019-04-15,S101AR07001-05-01,2019-04-15,625.0,1.0
63391,S101A,R070,01-05-01,ST. GEORGE,2019-04-17,626,2019-04-16,S101AR07001-05-01,2019-04-16,626.0,0.0
63392,S101A,R070,01-05-01,ST. GEORGE,2019-04-18,626,2019-04-17,S101AR07001-05-01,2019-04-17,626.0,0.0
63393,S101A,R070,01-05-01,ST. GEORGE,2019-04-19,626,2019-04-18,S101AR07001-05-01,2019-04-18,626.0,0.0


Drop the first date rows that have NaN entries

In [21]:
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [22]:
turnstiles_daily.head()

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,turnstile_id,PREV_DATE,PREV_ENTRIES,day_minus_prev_entries
1,A002,R051,02-00-00,59 ST,2019-03-31,6999957,2019-03-30,A002R05102-00-00,2019-03-30,6999064.0,893.0
2,A002,R051,02-00-00,59 ST,2019-04-01,7000528,2019-03-31,A002R05102-00-00,2019-03-31,6999957.0,571.0
3,A002,R051,02-00-00,59 ST,2019-04-02,7002087,2019-04-01,A002R05102-00-00,2019-04-01,7000528.0,1559.0
4,A002,R051,02-00-00,59 ST,2019-04-03,7003680,2019-04-02,A002R05102-00-00,2019-04-02,7002087.0,1593.0
5,A002,R051,02-00-00,59 ST,2019-04-04,7005332,2019-04-03,A002R05102-00-00,2019-04-03,7003680.0,1652.0


Sanity check that today's entries are greater than tomorrow's entries


In [23]:
try:
    assert (turnstiles_daily["ENTRIES"] > turnstiles_daily["PREV_ENTRIES"]).all()
except:
    print("assertion failed")

assertion failed


Let's look at the rows where yesterday's counts are greater than today's counts

In [24]:
yesterday_greater = turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]

yesterday_greater.head()

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,turnstile_id,PREV_DATE,PREV_ENTRIES,day_minus_prev_entries
1800,A025,R023,01-03-01,34 ST-HERALD SQ,2019-03-31,1238567092,2019-03-30,A025R02301-03-01,2019-03-30,1.238570e+09,-2753.0
1801,A025,R023,01-03-01,34 ST-HERALD SQ,2019-04-01,1238565030,2019-03-31,A025R02301-03-01,2019-03-31,1.238567e+09,-2062.0
1802,A025,R023,01-03-01,34 ST-HERALD SQ,2019-04-02,1238561212,2019-04-01,A025R02301-03-01,2019-04-01,1.238565e+09,-3818.0
1803,A025,R023,01-03-01,34 ST-HERALD SQ,2019-04-03,1238557159,2019-04-02,A025R02301-03-01,2019-04-02,1.238561e+09,-4053.0
1804,A025,R023,01-03-01,34 ST-HERALD SQ,2019-04-04,1238553023,2019-04-03,A025R02301-03-01,2019-04-03,1.238557e+09,-4136.0


In [25]:
yesterday_greater.groupby(['turnstile_id']).count().head(10)

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,PREV_DATE,PREV_ENTRIES,day_minus_prev_entries
turnstile_id,,,,,,,,,,
A025R02301-03-01,26,26,26,26,26,26,26,26,26,26
A038R08500-00-01,1,1,1,1,1,1,1,1,1,1
A042R08601-00-04,1,1,1,1,1,1,1,1,1,1
A047R08700-06-02,1,1,1,1,1,1,1,1,1,1
C021R21200-00-02,27,27,27,27,27,27,27,27,27,27
H003R16301-00-02,27,27,27,27,27,27,27,27,27,27
H023R23600-06-00,27,27,27,27,27,27,27,27,27,27
J034R00700-00-02,27,27,27,27,27,27,27,27,27,27
K026R10000-00-01,27,27,27,27,27,27,27,27,27,27


Many of these have 27 entries. Let's check out one

In [26]:
turnstiles_daily[turnstiles_daily.turnstile_id == 'C021R21200-00-02'].sort_values('datetime').head(10)

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,turnstile_id,PREV_DATE,PREV_ENTRIES,day_minus_prev_entries
9536,C021,R212,00-00-02,59 ST,2019-03-31,2029814690,2019-03-30,C021R21200-00-02,2019-03-30,2.029816e+09,-966.0
9537,C021,R212,00-00-02,59 ST,2019-04-01,2029814081,2019-03-31,C021R21200-00-02,2019-03-31,2.029815e+09,-609.0
9538,C021,R212,00-00-02,59 ST,2019-04-02,2029812726,2019-04-01,C021R21200-00-02,2019-04-01,2.029814e+09,-1355.0
9539,C021,R212,00-00-02,59 ST,2019-04-03,2029811072,2019-04-02,C021R21200-00-02,2019-04-02,2.029813e+09,-1654.0
9540,C021,R212,00-00-02,59 ST,2019-04-04,2029809414,2019-04-03,C021R21200-00-02,2019-04-03,2.029811e+09,-1658.0
9541,C021,R212,00-00-02,59 ST,2019-04-05,2029807782,2019-04-04,C021R21200-00-02,2019-04-04,2.029809e+09,-1632.0
9542,C021,R212,00-00-02,59 ST,2019-04-06,2029806237,2019-04-05,C021R21200-00-02,2019-04-05,2.029808e+09,-1545.0
9543,C021,R212,00-00-02,59 ST,2019-04-07,2029805316,2019-04-06,C021R21200-00-02,2019-04-06,2.029806e+09,-921.0
9544,C021,R212,00-00-02,59 ST,2019-04-08,2029804628,2019-04-07,C021R21200-00-02,2019-04-07,2.029805e+09,-688.0
9545,C021,R212,00-00-02,59 ST,2019-04-09,2029803065,2019-04-08,C021R21200-00-02,2019-04-08,2.029805e+09,-1563.0


In [27]:
turnstiles_daily.count()

CA                        60909
UNIT                      60909
SCP                       60909
STATION                   60909
DATE                      60909
ENTRIES                   60909
datetime                  60909
turnstile_id              60909
PREV_DATE                 60909
PREV_ENTRIES              60909
day_minus_prev_entries    60909
dtype: int64

In [28]:
turnstiles_daily[turnstiles_daily.day_minus_prev_entries < 0].count()

CA                        752
UNIT                      752
SCP                       752
STATION                   752
DATE                      752
ENTRIES                   752
datetime                  752
turnstile_id              752
PREV_DATE                 752
PREV_ENTRIES              752
day_minus_prev_entries    752
dtype: int64

What percentage of data do we lose if we through out the negative values?

In [29]:
752 / 60909

0.012346287084010574

We would lose 1.2%. I'm okay with that.
Eliminate negative values of day_minus_prev_entries

In [30]:
turnstiles_daily = turnstiles_daily[turnstiles_daily.day_minus_prev_entries > 0]
turnstiles_daily.head()

,CA,UNIT,SCP,STATION,DATE,ENTRIES,datetime,turnstile_id,PREV_DATE,PREV_ENTRIES,day_minus_prev_entries
1,A002,R051,02-00-00,59 ST,2019-03-31,6999957,2019-03-30,A002R05102-00-00,2019-03-30,6999064.0,893.0
2,A002,R051,02-00-00,59 ST,2019-04-01,7000528,2019-03-31,A002R05102-00-00,2019-03-31,6999957.0,571.0
3,A002,R051,02-00-00,59 ST,2019-04-02,7002087,2019-04-01,A002R05102-00-00,2019-04-01,7000528.0,1559.0
4,A002,R051,02-00-00,59 ST,2019-04-03,7003680,2019-04-02,A002R05102-00-00,2019-04-02,7002087.0,1593.0
5,A002,R051,02-00-00,59 ST,2019-04-04,7005332,2019-04-03,A002R05102-00-00,2019-04-03,7003680.0,1652.0


Looking at this data, the day_minus_prev_entries column tells us how many peole entered on a given day at a given turnstile.
Now, I'd like to group by station and sum over date to get total entries

In [38]:
turnstiles_daily.groupby('STATION').day_minus_prev_entries.sum().sort_values(ascending=False).head(20)

STATION
GRD CNTRL-42 ST    312007842.0
CARROLL ST         147348187.0
34 ST-PENN STA       2783315.0
34 ST-HERALD SQ      2756639.0
TIMES SQ-42 ST       2441108.0
59 ST COLUMBUS       1813923.0
23 ST                1637758.0
CHAMBERS ST          1360505.0
42 ST-PORT AUTH      1346362.0
47-50 STS ROCK       1318065.0
59 ST                1271166.0
50 ST                1236672.0
86 ST                1222671.0
JKSN HT-ROOSVLT      1215876.0
JAY ST-METROTEC       987458.0
42 ST-BRYANT PK       933867.0
14 ST                 922678.0
ATL AV-BARCLAY        909274.0
FLUSHING-MAIN         896817.0
LEXINGTON AV/53       884524.0
Name: day_minus_prev_entries, dtype: float64